## RandAugmentation_vanila
- baseline code가 존재하는 디렉토리에 해당 노트북을 다운받아 실행해주시기바랍니다.

In [1]:
import torch
import os
import sys
import yaml
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
from typing import Tuple
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

if "./" not in sys.path:
    sys.path.append("./")

In [2]:
from src.model import Model
from src.trainer import TorchTrainer
from src.loss import CustomCriterion

In [3]:
MEAN_V = (0.4914, 0.4822, 0.4465)
STD_V = (0.2470, 0.2435, 0.2616)

DATASET_DIR = "./input/cifar10"

In [4]:
def generate_transform(resize: int = 32, aug_fcns: Tuple = ()) -> transforms.transforms.Compose:
    """Generate train augmentation policy."""
    transform_fcns = []
    transform_fcns.append(transforms.Resize((resize, resize)))
    transform_fcns += list(aug_fcns)
    
    transform_fcns.append(transforms.ToTensor())
    transform_fcns.append(transforms.Normalize(MEAN_V, STD_V))
    
    return transforms.Compose(transform_fcns)
    

In [5]:
def load_cifar10(img_size: int = 32, 
                 aug_fcns: Tuple = (), 
                 validation_ratio: float = 0.8,
                ) -> Tuple[CIFAR10, CIFAR10, CIFAR10]:
    tf_train = generate_transform(resize=img_size, aug_fcns=aug_fcns)
    tf_test = generate_transform(resize=img_size)
    
    train_dataset = CIFAR10(root=DATASET_DIR, train=True, download=True, transform=tf_train)
    train_length = int(len(train_dataset) * validation_ratio)
    val_length = len(train_dataset) - train_length

    train_dataset, val_dataset = random_split(train_dataset, [train_length, val_length])
    test_dataset = CIFAR10(root=DATASET_DIR, train=False, download=True, transform=tf_test)
    
    return train_dataset, val_dataset, test_dataset
    

In [6]:
img_size = 32

augmentation_functions = (
    transforms.ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5), saturation=(0.5, 1.5)), 
    transforms.RandomPerspective(),
    transforms.RandomHorizontalFlip(),
)

train_dataset, val_dataset, test_dataset = load_cifar10(img_size=img_size, aug_fcns = augmentation_functions, validation_ratio=0.8)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./input/cifar10/cifar-10-python.tar.gz to ./input/cifar10
Files already downloaded and verified


In [7]:
def tensor_to_img(tensor_img: torch.Tensor) -> np.ndarray:
    return ((tensor_img.permute(1, 2, 0).numpy() * STD_V + MEAN_V) * 255).astype(np.uint8)

def visualize_datasets(_train_dataset: CIFAR10, _val_dataset: CIFAR10, _test_dataset: CIFAR10, title_prefix: str = "") -> None:
    fig, ax = plt.subplots(3, 7, figsize=(20, 10))

    for i in range(7):
        idx = np.random.randint(0, len(val_dataset))

        ax[0][i].imshow(tensor_to_img(_train_dataset[idx][0]))
        ax[1][i].imshow(tensor_to_img(_val_dataset[idx][0]))
        ax[2][i].imshow(tensor_to_img(_test_dataset[idx][0]))

        ax[0][i].axis('off')
        ax[1][i].axis('off')
        ax[2][i].axis('off')

    fig.suptitle(f"{title_prefix} Visualization of Augmentation.\n(Each row represents train, validation, test dataset accordingly)")
    fig.show()

In [8]:
visualize_datasets(train_dataset, val_dataset, test_dataset, title_prefix="Trial 00 //")

In [9]:
import yaml

In [10]:
with open("./configs/model/mobilenetv3.yaml", "r") as f:
    model_cfg = yaml.load(f, yaml.SafeLoader)

model_cfg['backbone'][-1][-1] = [10]

model = Model(model_cfg, verbose=True)

idx |   n |     params |          module |            arguments |   in_channel |   out_channel
----------------------------------------------------------------------------------------------
  0 |   1 |        464 |            Conv | [16, 3, 2, None, 1, 'HardSwish'] |            3           16
  1 |   1 |        464 | InvertedResidualv3 |  [3, 1, 16, 0, 0, 1] |           16           16
  2 |   1 |      3,440 | InvertedResidualv3 |  [3, 4, 24, 0, 0, 2] |           16           24
  3 |   1 |      4,440 | InvertedResidualv3 |  [3, 3, 24, 0, 0, 1] |           24           24
  4 |   1 |     10,328 | InvertedResidualv3 |  [5, 3, 40, 1, 0, 2] |           24           40
  5 |   1 |     20,992 | InvertedResidualv3 |  [5, 3, 40, 1, 0, 1] |           40           40
  6 |   1 |     20,992 | InvertedResidualv3 |  [5, 3, 40, 1, 0, 1] |           40           40
  7 |   1 |     32,080 | InvertedResidualv3 |  [3, 6, 80, 0, 1, 2] |           40           80
  8 |   1 |     34,760 | InvertedResidual

In [11]:
EPOCHS = 10
BATCH_SIZE = 256

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

optimizer = torch.optim.SGD(model.model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=0.1, steps_per_epoch=len(train_dataset), epochs=EPOCHS, pct_start=0.05)
criterion = CustomCriterion(samples_per_cls=None, device=device)

train_loader = DataLoader(dataset=train_dataset, 
                          pin_memory=torch.cuda.is_available(), 
                          shuffle=True, 
                          batch_size=BATCH_SIZE, 
                          num_workers=4, 
                          drop_last=True)
val_loader = DataLoader(dataset=val_dataset, 
                        pin_memory=torch.cuda.is_available(), 
                        shuffle=False, 
                        batch_size=BATCH_SIZE, 
                        num_workers=4)
test_loader = DataLoader(dataset=test_dataset, 
                         pin_memory=torch.cuda.is_available(), 
                         shuffle=False, 
                         batch_size=BATCH_SIZE, 
                         num_workers=4)
    
exp_dir = "./exp/autoaug"
os.makedirs(exp_dir, exist_ok=True)
trainer = TorchTrainer(model=model, 
                       criterion=criterion, 
                       optimizer=optimizer, 
                       scheduler=scheduler, 
                       device=device, 
                       verbose=1, 
                       model_path=os.path.join(exp_dir, "best.pt"))

best_acc, best_f1 = trainer.train(train_dataloader=train_loader, 
                                  n_epoch=EPOCHS, 
                                  val_dataloader=val_loader)

Train: [001] Loss: 2.156, Acc: 19.49% F1(macro): 0.19: 100%|██████████| 156/156 [00:10<00:00, 14.30it/s]
 Val:       Loss: 1.998, Acc: 26.18% F1(macro): 0.25: 100%|██████████| 40/40 [00:02<00:00, 15.40it/s]


Model saved. Current best test f1: 0.247


Train: [002] Loss: 1.860, Acc: 31.36% F1(macro): 0.30: 100%|██████████| 156/156 [00:10<00:00, 14.41it/s]
 Val:       Loss: 1.784, Acc: 34.64% F1(macro): 0.34: 100%|██████████| 40/40 [00:02<00:00, 16.56it/s]


Model saved. Current best test f1: 0.338


Train: [003] Loss: 1.726, Acc: 36.72% F1(macro): 0.36: 100%|██████████| 156/156 [00:10<00:00, 14.62it/s]
 Val:       Loss: 1.687, Acc: 38.66% F1(macro): 0.38: 100%|██████████| 40/40 [00:02<00:00, 14.54it/s]


...

Model saved. Current best test f1: 0.461


Train: [009] Loss: 1.426, Acc: 48.55% F1(macro): 0.48: 100%|██████████| 156/156 [00:10<00:00, 14.54it/s]
 Val:       Loss: 1.468, Acc: 47.35% F1(macro): 0.47: 100%|██████████| 40/40 [00:02<00:00, 16.07it/s]


Model saved. Current best test f1: 0.473


Train: [010] Loss: 1.410, Acc: 49.49% F1(macro): 0.49: 100%|██████████| 156/156 [00:10<00:00, 14.47it/s]
 Val:       Loss: 1.431, Acc: 48.15% F1(macro): 0.47: 100%|██████████| 40/40 [00:02<00:00, 15.80it/s]


Model saved. Current best test f1: 0.475


In [12]:
test_loss, test_f1, test_accuracy = trainer.test(model, test_loader)

 Val:       Loss: 1.320, Acc: 53.17% F1(macro): 0.52: 100%|██████████| 40/40 [00:00<00:00, 40.17it/s]
